<a href="https://colab.research.google.com/github/geek007git/gdp-predict/blob/main/country_forecast_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/gdp.csv")
print(df.head())

       Country    2020      2021      2022      2023      2024      2025
0  Afghanistan   20136   14278.0   14501.0   17248.0       NaN       NaN
1      Albania   15271   18086.0   19185.0   23388.0   27259.0   28372.0
2      Algeria  164774  185850.0  225709.0  247789.0  264913.0  268885.0
3      Andorra    2885    3325.0    3376.0    3786.0    4038.0    4035.0
4       Angola   66521   84375.0  142442.0  109764.0  115946.0  113343.0


In [ ]:
df_long = df.melt(id_vars="Country", var_name="Year", value_name="Value")
df_long["Year"] = df_long["Year"].astype(int)
print(df_long.head())


       Country  Year     Value
0  Afghanistan  2020   20136.0
1      Albania  2020   15271.0
2      Algeria  2020  164774.0
3      Andorra  2020    2885.0
4       Angola  2020   66521.0


In [ ]:

df_long["Value"] = df_long.groupby("Country")["Value"].transform(lambda x: x.interpolate())


In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

future_years = np.arange(2026, 2031).reshape(-1, 1)  # 2026 → 2030
forecasts = {}

for country in df["Country"]:
    country_data = df_long[df_long["Country"] == country].dropna()

    X = country_data["Year"].values.reshape(-1, 1)
    y = country_data["Value"].values

    model = LinearRegression()
    model.fit(X, y)

    preds = model.predict(future_years)
    forecasts[country] = preds


forecast_df = pd.DataFrame(forecasts, index=range(2026, 2031))
print(forecast_df.head())


       Afghanistan       Albania        Algeria      Andorra         Angola  \
2026  16498.200000  31649.533333  304302.400000  4404.066667  135013.000000   
2027  16418.685714  34427.447619  326583.085714  4641.180952  143474.285714   
2028  16339.171429  37205.361905  348863.771429  4878.295238  151935.571429   
2029  16259.657143  39983.276190  371144.457143  5115.409524  160396.857143   
2030  16180.142857  42761.190476  393425.142857  5352.523810  168858.142857   

      Antigua and Barbuda      Argentina       Armenia        Aruba  \
2026          2595.466667  771769.133333  31080.066667  4550.066667   
2027          2790.123810  827285.790476  34154.752381  4878.895238   
2028          2984.780952  882802.447619  37229.438095  5207.723810   
2029          3179.438095  938319.104762  40304.123810  5536.552381   
2030          3374.095238  993835.761905  43378.809524  5865.380952   

         Australia  ...  United States        Uruguay     Uzbekistan  \
2026  1.924333e+06  ...   

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


country_dropdown = widgets.Dropdown(
    options=sorted(df["Country"].unique()),
    description="Country:",
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)


def plot_forecast(country):
    past = df_long[df_long["Country"] == country]

    plt.figure(figsize=(8,5))
    plt.plot(past["Year"], past["Value"], marker="o", label=f"{country} (Actual)")
    plt.plot(forecast_df.index, forecast_df[country], marker="x", linestyle="--", label=f"{country} (Forecast)")

    plt.xlabel("Year")
    plt.ylabel("Value")
    plt.title(f"Forecast for {country} (2020–2030)")
    plt.legend()
    plt.show()


widgets.interact(plot_forecast, country=country_dropdown)
print("all the value is in USD")


interactive(children=(Dropdown(description='Country:', layout=Layout(width='400px'), options=('Afghanistan', '…

all the value is in USD
